# Altair examples
This notebook demonstrates how to create various plots in Altair. It accompanies [this Medium post]()

In [ ]:
import os

import altair as alt
import pandas as pd

Prepare the data

In [ ]:
map_path = os.path.join("..", "data", "boundaries", "uk_hex.csv")
data_path = os.path.join("..", "data", "map", "ge_2017.csv")

In [ ]:
map_df = pd.read_csv(map_path, encoding="latin-1")
data_df = pd.read_csv(data_path, encoding="latin-1")

In [ ]:
data_df = data_df[data_df.loc[:, "Year"] == 2017]
data_df = data_df.drop([col for col in data_df.columns if "Unnamed" in col], axis=1)

In [ ]:
data_df = data_df.drop(["Year", "Region", "Party Abbreviation", "Share of Vote", "Candidate Votes", "Code"], axis=1)
data_df = data_df.dropna()

In [ ]:
df = data_df.merge(map_df, on="Constituency")

In [ ]:
df["q"] = df.apply(lambda row: row.q if row.r % 2 == 0 else row.q + 0.5, axis=1)

## Basic Map
Create a hexmap. No colours. No interactivity.

In [ ]:
alt.Chart(df).mark_circle().encode(
        x="q",
        y="r",
        color=alt.value("lightgray"),
        size=alt.value(50),
    )

## Colour map
Colour each constituency by the winning party

In [ ]:
df["Party"].unique()

Define a colour mapping

In [ ]:
parties = ["Conservative", "Labour", "Lib Dem", "Green", "Scottish National Party",
          "Plaid Cymru", "Sinn Fein", "Speaker", "Democratic Unionist Party",
          "Independent"]
party_colours = ["darkblue", "red", "orange", "green", "yellow",
                "#98df8a", "darkgreen", "lightgray", "#ad494a", "gold"]

In [ ]:
colours_obj = alt.Color("Party:N", scale=alt.Scale(domain=parties, range=party_colours), legend=None)

In [ ]:
hexmap = alt.Chart(df).mark_circle().encode(
        x="q",
        y="r",
        color=colours_obj,
        size=alt.value(50),
    )

legend = alt.Chart(df).mark_circle().encode(
            y=alt.Y("Party:N", axis=alt.Axis(orient="right")), color=colours_obj
        )

hexmap | legend

## Basic interactivity
Add a tooltip for extra information

In [ ]:
alt.Chart(df).mark_circle().encode(
        x="q",
        y="r",
        color=colours_obj,
        size=alt.value(50),
        tooltip=["Constituency:N"],
    ).interactive()

## More interactivity
Highlight all constituencies belonging to a selected party

Create the selection object

In [ ]:
selector = alt.selection_single(empty='all', fields=['Party'])

Creat the conditional colouring

In [ ]:
colours_condition = alt.condition(selector, colours_obj, alt.value("white"))

Create the map and legend

In [ ]:
hexmap = alt.Chart(df).mark_circle().encode(
        x="q",
        y="r",
        color=colours_condition,
        size=alt.value(50),
        tooltip=["Constituency:N"],
    ).interactive().add_selection(selector)

legend = alt.Chart(df).mark_circle().encode(
            y=alt.Y("Party:N", axis=alt.Axis(orient="right")), color=colours_obj
        ).add_selection(selector)

hexmap | legend

Click on a circle, either in the map or the legend, to see the interactivity

## Adding another chart

In [ ]:
df["threshold"] = 325.

In [ ]:
bars = alt.Chart(df).mark_bar().encode(
            x="Party:N",
            y=alt.Y("count()", title="Number of MPs"),
            color=colours_condition
        ).add_selection(selector)

majority = alt.Chart(df).mark_rule(color="black", strokeDash=[1, 1]).encode(
    y="threshold:Q",
    size=alt.value(3)
)

legend | hexmap | (bars + majority)